In [ ]:
import pandas as pd
import os
import numpy as np
import re
import dendropy

In [ ]:
output_dir = '/Users/clairedubin/spur/peroxisomes/PAML_output/paml_output_031620/'

In [ ]:
df = pd.DataFrame()

for _, file in enumerate(os.listdir(output_dir)):
    
    
#     print(_, file)
    
    #find line with branch lengths
    
    lengths = {}
    
    f = open(output_dir+file, 'r')
    
    tree_next = False
    
    for line in f.readlines():
                
        if 'tree length = ' in line:
            
            tree_length = float(line.split('=')[1].strip(' ').strip('\n'))
            
            lengths['tree_length'] = tree_length
            
            tree_next = True
            
        elif tree_next:
            
            if re.match(r'^\(+[a-zA-Z]+', line):
                
                tree = dendropy.Tree.get(
                data=line,
                schema='newick',
                )
                
                tn = [str(i) for i in tree.taxon_namespace]
                
                dist = tree.calc_node_root_distances(return_leaf_distances_only=True)
                
                #drop genes with data for less than 7 strains
                

                if len(dist) >= 7:
                
                    #section below is redundant with the node iteration further below
#                     for i, sp in enumerate(tree.taxon_namespace):

#                         start = '_root'

#                         sp = str(sp).replace(' ', '_').replace("'", '') + '_root'

#                         lengths[sp] = [dist[i]]


                    sacc = [sp for sp in tree.taxon_namespace  if "Saccharomyces" in str(sp)]


                    edge_iter = tree.preorder_edge_iter()

                    for i in tree.preorder_node_iter():

                            edge = next(edge_iter)

                            node_label = '_'.join(sorted([str(a.taxon).replace(' ', '_').replace("'", '') for a in i.leaf_nodes()]))                    

                            edge_label = 'branch_to_' + node_label

                            #get length of subtree formed by this node

                            temp = tree.extract_tree_with_taxa([a.taxon for a in i.leaf_nodes()])
                            
                            lengths['subtree_'+node_label] = [temp.length()]

                            #length of edge leading to subtree, will be 0 for root

                            lengths[edge_label] = [edge.length]

                    temp_df = pd.DataFrame.from_dict(lengths)
                    temp_df.index=[file]
                    df = df.append(temp_df) 
                    
                else:
                    
                    print(dist)

                break

In [ ]:
#drop columns with more than 80% NaN

df = df.dropna(axis=1, thresh=df.shape[0]*.8)

df.head()

In [ ]:
#normalize to tree length

for col in df.columns:
    
    if col != 'tree_length':
        df[col] = df[col]/df['tree_length']
        
df    

In [ ]:
go5778 = ['YPL112C',
 'YHR150W',
 'YPR165W',
 'YGR028W',
 'YNL214W',
 'YPR128C',
 'YML075C',
 'YGR004W',
 'YGR239C',
 'YLR324W',
 'YGL153W',
 'YHR160C',
 'YMR163C',
 'YDR265W',
 'YLR450W',
 'YDR244W',
 'YMR026C',
 'YKL188C',
 'YDR329C',
 'YLR191W',
 'YDR479C',
 'YOL147C',
 'YOR193W',
 'YBR168W',
 'YPL147W',
 'YBR222C',
 'YNL329C',
 'YDL065C',
 'YOL044W',
 'YAL055W',
'YJL210W',
'YKL197C',
'YMR018W']

In [ ]:
#change index from file name to gene name

df.index = df.reset_index()['index'].str.split('_', expand=True)[0]

In [ ]:
go5778_df = df[df.index.isin(go5778)]

go5778_df

In [ ]:
essential = pd.read_csv('/Users/clairedubin/spur/publishable_data/external_datasets/essential.csv', header=None)
essential[1] = essential[1].str.strip('\t')
essential_genes = essential[1].str.strip(' ').tolist()

In [ ]:
def resample(df1, df2, cols='all', n=10000):
    
    results_dict = {}
    
    
    if cols != 'all':

        df1 = df1[cols]
        df2 = df2[cols]

    actual_meds = df1.median(axis=0)
    
    essential_count = len([i for i in df1.index if i in essential_genes])
    nonessential_count = len(df1.index) - essential_count
    
    print('essential count: ', essential_count)
    print('nonessential_count: ', nonessential_count)
    
    essential_df = df2[df2.index.isin(essential_genes)]
    nonessential_df =  df2[~df2.index.isin(essential_genes)]
        
    for i in range(len(df1.columns)):
        results_dict[i] = 0

    for _ in range(n):
        
#         if _ % 100 == 0:
        
#             print(_)

        sample = essential_df.sample(n=essential_count)
        sample = sample.append(nonessential_df.sample(n=nonessential_count))

        sample_meds = sample.median(axis=0)

        for i, val in enumerate(actual_meds):

            if sample_meds[i] >= val:

                results_dict[i] += 1

    results = {}

    for i in results_dict:

        results[df1.columns[i]] = results_dict[i]/n

    return results        
    

        

In [ ]:
np.random.seed(222)

results = resample(go5778_df, df)

In [ ]:
r = pd.DataFrame.from_dict(results, orient='index').rename(columns={0:'p'})
r

In [ ]:
#significant values
r[r['p'] < 0.06]